In [1]:
import matplotlib
import pandas as pd
import requests
import io
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
import re
from datetime import datetime


%matplotlib inline
    


%matplotlib inline
url1 = "https://raw.githubusercontent.com/docju/datasciencecapstone/main/results.csv"
download1 = requests.get(url1).content

df = pd.read_csv(io.StringIO(download1.decode('ISO-8859-1')))
df['date']=pd.to_datetime(df['DateTime'])
df['outcome']=df['FTHG']-df['FTAG']

def result(n):
    if n >0:
        res=1
    if n==0:
        res=0
    if n<0:
        res=-1
    return res

df['result']=df.apply(lambda x: result(x['outcome']),axis=1)
all_results=df[['Season','date','HomeTeam','AwayTeam','FTHG','FTAG','outcome','result']][~df['Season'].isin(['2020-21','2021-22'])]

print(all_results.head())

    Season       date     HomeTeam        AwayTeam  FTHG  FTAG  outcome  \
0  1993-94 1993-08-14      Arsenal        Coventry     0     3       -3   
1  1993-94 1993-08-14  Aston Villa             QPR     4     1        3   
2  1993-94 1993-08-14      Chelsea       Blackburn     1     2       -1   
3  1993-94 1993-08-14    Liverpool  Sheffield Weds     2     0        2   
4  1993-94 1993-08-14     Man City           Leeds     1     1        0   

   result  
0      -1  
1       1  
2      -1  
3       1  
4       0  


In [2]:
#keep most recent entry

#all_results.groupby(["HomeTeam","AwayTeam"], as_index=False).max()
teams=all_results[all_results['Season']!='2019-20']['HomeTeam'].unique()

#keep only most recent entry
most_recent=all_results[all_results['Season']!='2019-20'].sort_values('date').drop_duplicates(['HomeTeam','AwayTeam'],keep='last')
#keep average of all previous results
average_results=all_results[all_results['Season']!='2019-20'].groupby(['HomeTeam','AwayTeam']).mean()['result'].reset_index()

most_recent_matrix=np.asmatrix(average_results.groupby(['HomeTeam', 'AwayTeam'])['result'].max().unstack().fillna(-2))

def table_create(scores=most_recent):
    #period=scores[all_results['Season']==season].copy()
    period=scores.copy()
    period['home_points']=period.apply(lambda x: result_1(x['result']),axis=1)
    period['away_points']=period.apply(lambda x: result_1(-1*x['result']),axis=1)
    period['home_gd']=period['outcome']
    period['away_gd']=-1*period['outcome']
    period['home_games']=1
    period['away_games']=1
    home_games=period.groupby('HomeTeam').sum()['home_games']
    away_games=period.groupby('AwayTeam').sum()['away_games']
    home_sum=period.groupby('HomeTeam').sum()['home_points']
    away_sum=period.groupby('AwayTeam').sum()['away_points']
    home_gd=period.groupby('HomeTeam').sum()['home_gd']
    away_gd=period.groupby('AwayTeam').sum()['away_gd']
    home_goals=period.groupby('HomeTeam').sum()['FTHG']
    away_goals=period.groupby('AwayTeam').sum()['FTAG']
    points_total=pd.concat([home_games,away_games,home_sum,away_sum, home_gd,away_gd,home_goals,away_goals],axis=1)
    points_total.index.name='Team'
    points_total=points_total.reset_index()
    points_total['Played']=points_total['home_games']+points_total['away_games']
    points_total['GF']=points_total['FTHG']+points_total['FTAG']
    points_total['GD']=points_total['home_gd']+points_total['away_gd']
    points_total['Points']=points_total['home_points']+points_total['away_points']
    points_total=points_total[['Played','Team','GF','GD','Points']].sort_values(['Points','GD','GF','Team'],ascending=[False,False,False,True])   
    
    return points_total


def split_season_train_test (test,input_df):
    '''
    This function takes data from a particular season and splits it into training and test samples
    
    INPUT:
    input_df - (dataframe) input matrix containing results of all seasons
    season - (string) season to split
    test - (int) number of rows that constitute the test data frame
    
    OUTPUT:
    season_train - (dataframe) a dataframe containing only the non-test rows
    season_test - (dataframe) a dataframe containing the test rows
    '''
    train=input_df.shape[0]-test
    season_train=input_df.copy().head(train)
    season_test=input_df.copy().tail(test)

    return season_train, season_test

train,test=split_season_train_test(92,most_recent)